In [211]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# from sklearn:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer #should use KNNImputer??!
from sklearn.impute import KNNImputer
from sklearn.tree import DecisionTreeClassifier #plant forests, not trees?!
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder #should use Ordinal??!
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler #should use Quantile??!
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor #check documentation??!
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

In [212]:
housing_classification = pd.read_csv(r'housing-classification.csv')
test_housing_classification = pd.read_csv(r'test-housing-classification.csv')

In [213]:
housing_classification.set_index('Id', inplace=True)
test_housing_classification.set_index('Id', inplace=True)

In [214]:
housing_classification.drop(housing_classification.select_dtypes(['object']).columns, axis=1, inplace=True)

In [215]:
expensive_houses = housing_classification.pop('Expensive') 
X = housing_classification.copy()
y = expensive_houses.copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)

In [216]:
scaler = QuantileTransformer(output_distribution='normal')

In [217]:
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [218]:
imputer = KNNImputer(missing_values=np.nan, n_neighbors=5).set_output(transform='pandas')
imputer.fit(X_train)
X_imputed_train = imputer.transform(X_train_scaled)
X_imputed_test = imputer.transform(X_test_scaled)

c:\Users\AlexD\Desktop\Python\Python 3.10.6\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNNImputer was fitted with feature names
  warnings.warn(
c:\Users\AlexD\Desktop\Python\Python 3.10.6\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNNImputer was fitted with feature names
  warnings.warn(


In [219]:
imputed_tree = DecisionTreeClassifier(max_depth=10,
                                        min_samples_leaf=4)
imputed_tree.fit(X=X_imputed_train,
                y= y_train)
y_imputed_pred = imputed_tree.predict(X_imputed_train)
accuracy_score(y_true = y_train,
                y_pred = y_imputed_pred)

0.9751712328767124

2. on test:

In [220]:
test_housing_classification.drop(test_housing_classification.select_dtypes(['object']).columns, axis=1, inplace=True) 
test_housing_classification_scaled = scaler.transform(test_housing_classification)
X_imputed_test = imputer.transform(test_housing_classification_scaled)
y_imputed_pred_test = imputed_tree.predict(X_imputed_test)
test_housing_classification["Expensive"] = y_imputed_pred_test

c:\Users\AlexD\Desktop\Python\Python 3.10.6\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but KNNImputer was fitted with feature names
  warnings.warn(


In [221]:
test3 = test_housing_classification["Expensive"]
# test2.sample(25)
test3.to_csv('3rd_test')